In [51]:
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.dialects.mysql import insert
import os
import pandas as pd

## Load experimental gene expression data
First, establish a connection

In [46]:
load_dotenv()
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASS')
db_port = 10184
database = "defaultdb"
host_url = rf"mysql://{db_user}:{db_password}@{db_host}:{db_port}/{database}"

engine = create_engine(host_url)

Now, gather data from all files

In [9]:
with open('DD_genes.txt', 'r') as f:
    genes_DD = f.read().splitlines()

with open('LD_genes.txt', 'r') as f:
    genes_LD = f.read().splitlines()
    
print(len(genes_DD))
print(len(genes_LD))
long_gene = max(set(genes_DD).union(set(genes_LD)), key=lambda x: len(x))
print(f"{long_gene}, lenght={len(long_gene)}")

15650
15743
alphagamma-element:CR32865, lenght=26


In [52]:
def insert_on_duplicate(table, conn, keys, data_iter):
    insert_stmt = insert(table.table).values(list(data_iter))
    on_duplicate_key_stmt = insert_stmt.on_duplicate_key_update(insert_stmt.inserted)
    conn.execute(on_duplicate_key_stmt)

In [80]:
DATA_PATH = r'../dataset/'
mapper = lambda x: x[1:]  # removes leading 'x' char in idx strings
list_DD = []
list_LD = []
file_names = os.listdir(DATA_PATH)

for filename in file_names:
    # Read data from csv
    print(f"filename: {filename}")
    table_name = filename.split('.')[0]
    new_df = pd.read_csv(f"{DATA_PATH}/{filename}", keep_default_na=False)
    new_df = new_df.rename(mapper, axis='columns')
    new_df = new_df.rename({'nnamed: 0': 'gene'}, axis='columns')
    
    assert len(genes_DD) == len(new_df['gene'].unique()) == len(new_df)
    # # Generate sql queries
    ddl = pd.io.sql.get_schema(new_df, 'data')

    drop_table_query = f"""
    DROP TABLE IF EXISTS {table_name} CASCADE;"""

    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        gene VARCHAR(30) COLLATE utf8_bin PRIMARY KEY, 
    {'\n'.join(ddl.split('\n')[2:])};
    """
    # print(create_table_query)

    # Create table
    with engine.connect() as connection:
        connection.execute(text(drop_table_query))
        connection.execute(text(create_table_query))

    new_df.to_sql(table_name, engine, if_exists='append', index=False)

# df_LD = pd.concat(list_LD)
# df_DD = pd.concat(list_DD

filename: GSM4768020_CT02_20190528_AR05.csv
filename: GSM4768021_CT02_20190528_AR06.csv
filename: GSM4768022_CT02_20190528_AR07.csv
filename: GSM4768023_CT02_20190528_AR08.csv
filename: GSM4768024_CT02_20190702_AR13.csv
filename: GSM4768025_CT02_20190702_AR14.csv
filename: GSM4768026_CT02_20190702_AR15.csv
filename: GSM4768027_CT02_20190702_AR16.csv
filename: GSM4768028_CT06_20190710_AR01.csv
filename: GSM4768029_CT06_20190710_AR02.csv
filename: GSM4768030_CT06_20190710_AR03.csv
filename: GSM4768031_CT06_20190710_AR04.csv
filename: GSM4768032_CT06_20190719_AR17.csv
filename: GSM4768033_CT06_20190719_AR18.csv
filename: GSM4768034_CT06_20190814_AR19.csv
filename: GSM4768035_CT06_20190814_AR20.csv
filename: GSM4768036_CT10_20190524_AR03.csv
filename: GSM4768037_CT10_20190524_AR04.csv
filename: GSM4768038_CT10_20190610_AR01.csv
filename: GSM4768039_CT10_20190610_AR02.csv
filename: GSM4768040_CT10_20190704_AR04.csv
filename: GSM4768041_CT10_20190704_AR05.csv
filename: GSM4768042_CT10_201907

AssertionError: 

In [79]:
pd.read_sql("""SELECT * FROM GSM4768020_CT02_20190528_AR05 WHERE gene in ('amon', 'dpr8'); """, con=engine)

,gene,20190528_CLK856_DD_CT02_AR05_ACAGAC,20190528_CLK856_DD_CT02_AR05_ACAGGA,20190528_CLK856_DD_CT02_AR05_ACAGTG,20190528_CLK856_DD_CT02_AR05_ACCAAC,20190528_CLK856_DD_CT02_AR05_ACCAGA,20190528_CLK856_DD_CT02_AR05_ACCATG,20190528_CLK856_DD_CT02_AR05_ACGTAC,20190528_CLK856_DD_CT02_AR05_ACGTGA,20190528_CLK856_DD_CT02_AR05_ACGTTG,...,20190528_CLK856_DD_CT02_AR05_TGAGTG,20190528_CLK856_DD_CT02_AR05_TGCAAC,20190528_CLK856_DD_CT02_AR05_TGCAGA,20190528_CLK856_DD_CT02_AR05_TGCATG,20190528_CLK856_DD_CT02_AR05_TGGTAC,20190528_CLK856_DD_CT02_AR05_TGGTGA,20190528_CLK856_DD_CT02_AR05_TGGTTG,20190528_CLK856_DD_CT02_AR05_TGTCAC,20190528_CLK856_DD_CT02_AR05_TGTCGA,20190528_CLK856_DD_CT02_AR05_TGTCTG
0,amon,35,6,1,2,0,1,3,0,0,...,9,6,3,1,1,0,168,51,3,22
1,dpr8,27,5,0,1,0,2,1,0,0,...,9,3,1,22,0,0,14,8,5,31


In [ ]:
with engine.connect() as connection:
    connection.execute(text("COMMIT"))

In [13]:
pd.read_sql('SHOW TABLES', engine)

,Tables_in_defaultdb
0,GSM4768020_CT02_20190528_AR05
1,GSM4768021_CT02_20190528_AR06
2,GSM4768022_CT02_20190528_AR07
3,GSM4768023_CT02_20190528_AR08
4,GSM4768024_CT02_20190702_AR13
...,...
80,GSM4768100_ZT22_20190219_AR12
81,GSM4768101_ZT22_20190228_AR012
82,GSM4768102_ZT22_20190309_AR01
83,GSM4768103_ZT22_20190309_AR02


In [14]:
table = 'GSM4768022_CT02_20190528_AR07'
pd.read_sql(f"describe {table}", engine)

,Field,Type,Null,Key,Default,Extra
0,gene,varchar(30),NO,PRI,None,
1,20190528_CLK856_DD_CT02_AR07_ACAGAC,int,YES,,None,
2,20190528_CLK856_DD_CT02_AR07_ACAGGA,int,YES,,None,
3,20190528_CLK856_DD_CT02_AR07_ACAGTG,int,YES,,None,
4,20190528_CLK856_DD_CT02_AR07_ACCAAC,int,YES,,None,
...,...,...,...,...,...,...
92,20190528_CLK856_DD_CT02_AR07_TGGTGA,int,YES,,None,
93,20190528_CLK856_DD_CT02_AR07_TGGTTG,int,YES,,None,
94,20190528_CLK856_DD_CT02_AR07_TGTCAC,int,YES,,None,
95,20190528_CLK856_DD_CT02_AR07_TGTCGA,int,YES,,None,
